In [51]:
import ccxt
import datetime
import time
from pathlib import Path
import csv

In [52]:
SECONDS_IN_HOUR = 60*60
SECONDS_IN_DAY = SECONDS_IN_HOUR*24
SECONDS_IN_MONTH = SECONDS_IN_DAY*30
SECONDS_IN_YEAR = SECONDS_IN_DAY*365

In [117]:
start_time = datetime.datetime(2020, 3, 1, 0, 0)
start_time_unix = time.mktime(start_time.timetuple())
start_time_unix_ms = start_time_unix * 1000

In [119]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [130]:
scrape_candles_to_csv('btc_usd_1h.csv', 'ftx', 100, 'BTC/USD', '1h', '2020-01-0100:00:00z', 5000)

5000 BTC/USD candles in total from 2021-04-25T20:00:00.000Z to 2021-11-20T03:00:00.000Z
10000 BTC/USD candles in total from 2020-09-29T12:00:00.000Z to 2021-11-20T03:00:00.000Z
15000 BTC/USD candles in total from 2020-03-05T04:00:00.000Z to 2021-11-20T03:00:00.000Z
20000 BTC/USD candles in total from 2019-08-09T20:00:00.000Z to 2021-11-20T03:00:00.000Z
Saved 20000 candles from 2019-08-09T20:00:00.000Z to 2021-11-20T03:00:00.000Z to btc_usd_1h.csv
